In [1]:
!pip -q install -U transformers datasets accelerate evaluate scikit-learn pyarrow py_vncorenlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.31.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-adk 1.21.0 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.


In [2]:
import os, json, re, unicodedata, random, warnings, gc
import numpy as np
import pandas as pd
import torch

warnings.filterwarnings("ignore", category=FutureWarning)

SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ============================================================
#  CHỌN MODEL: comment/uncomment để chuyển backbone
# ============================================================
# MODEL_NAME = "xlm-roberta-base"         # Option A: không cần word-seg
MODEL_NAME = "vinai/phobert-base-v2"       # Option B: mạnh hơn, cần word-seg

USE_WORD_SEG = MODEL_NAME.startswith("vinai/phobert")

MAX_LENGTH   = 256
EPOCHS       = 10
LR           = 2e-5
TRAIN_BS     = 16       # per-GPU
EVAL_BS      = 64
GRAD_ACCUM   = 2        # effective BS = 16×2×2GPU = 64
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
PATIENCE     = 3
DROPOUT_RATE = 0.2

VISP_SAMPLE_SIZE = 50000
VIQP_ENABLED     = True

DO_HARD_NEG_MINING = True
HARD_NEG_PER_SENT  = 2
HARD_NEG_WEIGHT    = 0.5

PREP_DIR = "/kaggle/working/vnpi_prepared"
OUT_DIR  = "/kaggle/working/vnpi_model"
os.makedirs(PREP_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

print(f"Model: {MODEL_NAME} | Word-seg: {USE_WORD_SEG}")
print(f"PyTorch: {torch.__version__} | CUDA: {torch.version.cuda} | GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"  [{i}] {torch.cuda.get_device_name(i)}")

Model: vinai/phobert-base-v2 | Word-seg: True
PyTorch: 2.9.0+cu126 | CUDA: 12.6 | GPUs: 2
  [0] Tesla T4
  [1] Tesla T4


In [3]:
import py_vncorenlp
from datasets import load_dataset
from transformers import AutoTokenizer

rdrsegmenter = None
if USE_WORD_SEG:
    WSEG_DIR = "/kaggle/working/vncorenlp"
    os.makedirs(WSEG_DIR, exist_ok=True)
    py_vncorenlp.download_model(save_dir=WSEG_DIR)
    rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=WSEG_DIR)
    print("VnCoreNLP word segmenter loaded ✓")
else:
    print("Word segmentation: disabled (XLM-R mode)")

def norm_text(s) -> str:
    s = "" if s is None else str(s)
    s = unicodedata.normalize("NFC", s)
    return re.sub(r"\s+", " ", s).strip()

def word_segment(text: str) -> str:
    if not rdrsegmenter or not text.strip(): return text
    try: return " ".join(rdrsegmenter.word_segment(text))
    except: return text

def process_text(s) -> str:
    s = norm_text(s)
    return word_segment(s) if USE_WORD_SEG else s

def canonical_pair(a, b):
    return (a, b) if a <= b else (b, a)

--2026-02-20 16:55:33--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412703 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.2.jar’

     0K .......... .......... .......... .......... ..........  0% 3.46M 8s
    50K .......... .......... .......... .......... ..........  0% 4.76M 7s
   100K .......... .......... .......... .......... ..........  0% 17.7M 5s
   150K .......... .......... .......... .......... ..........  0% 25.5M 4s
   200K .......... .......... .......... .......... ..........  0% 6.45M 4s
   250K .......... .......... .......... .......... ..........  1% 48.7M 3s
   300K .......... .......... .......... .......... ..........  1% 21.8M 3s
   350K ..

2026-02-20 16:55:37 INFO  WordSegmenter:24 - Loading Word Segmentation model
VnCoreNLP word segmenter loaded ✓


In [4]:
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# ---- 3.1) VNPC + vnPara ----
print("Loading VNPC + vnPara...")
ds_vnpc  = load_dataset("chienpham/vnpc-train", split="train")
ds_vnpara = load_dataset("chienpham/vnpara", split="train")

df_vnpc = ds_vnpc.to_pandas(); df_vnpc["source"] = "vnpc"
df_vnpara = ds_vnpara.to_pandas(); df_vnpara["source"] = "vnpara"
if "score" in df_vnpara.columns and "label" not in df_vnpara.columns:
    df_vnpara = df_vnpara.rename(columns={"score": "label"})
if "Unnamed: 0" in df_vnpara.columns:
    df_vnpara = df_vnpara.drop(columns=["Unnamed: 0"])

print(f"  vnpc:   {len(df_vnpc)}, labels={df_vnpc['label'].value_counts().to_dict()}")
print(f"  vnpara: {len(df_vnpara)}, labels={df_vnpara['label'].value_counts().to_dict()}")

# ---- 3.2) ViSP (clone GitHub) ----
print(f"\nLoading ViSP (sampling {VISP_SAMPLE_SIZE})...")
VISP_DIR = "/kaggle/working/ViSP"
if not os.path.exists(VISP_DIR):
    os.system(f"git clone --depth 1 https://github.com/ngwgsang/ViSP.git {VISP_DIR}")

visp_files = []
for root, dirs, files in os.walk(VISP_DIR):
    for f in files:
        if f.endswith(('.json', '.jsonl')):
            visp_files.append(os.path.join(root, f))
print(f"  Found: {[os.path.basename(f) for f in visp_files]}")

visp_pairs = []
for fpath in visp_files:
    try:
        with open(fpath, 'r', encoding='utf-8') as fp:
            content = fp.read().strip()
            items = json.loads(content) if content.startswith('[') else \
                    [json.loads(l) for l in content.split('\n') if l.strip()]
        for item in items:
            orig = item.get("original", "")
            for p in item.get("paraphrases", []):
                pt = p.get("text", p) if isinstance(p, dict) else str(p)
                if orig and pt:
                    visp_pairs.append({"sentence1":orig,"sentence2":pt,"label":1,"source":"visp"})
    except Exception as e:
        print(f"  Warning: {os.path.basename(fpath)}: {e}")

print(f"  Total ViSP pairs: {len(visp_pairs)}")
if len(visp_pairs) > VISP_SAMPLE_SIZE:
    random.shuffle(visp_pairs)
    visp_pairs = visp_pairs[:VISP_SAMPLE_SIZE]
    print(f"  Sampled to: {len(visp_pairs)}")
df_visp = pd.DataFrame(visp_pairs) if visp_pairs else pd.DataFrame(
    columns=["sentence1","sentence2","label","source"])

# ---- 3.3) ViQP (HuggingFace) ----
if VIQP_ENABLED:
    print("\nLoading ViQP...")
    try:
        ds_viqp = load_dataset("SCM-LAB/ViQP")
        viqp_pairs = []
        for split_name in ds_viqp:
            for item in ds_viqp[split_name]:
                sq = item.get("source","")
                targets = item.get("target",[])
                if isinstance(targets, str): targets = [targets]
                for t in targets:
                    if sq and t:
                        viqp_pairs.append({"sentence1":sq,"sentence2":t,"label":1,"source":"viqp"})
        df_viqp = pd.DataFrame(viqp_pairs)
        print(f"  ViQP pairs: {len(df_viqp)}")
    except Exception as e:
        print(f"  ViQP failed: {e}")
        df_viqp = pd.DataFrame(columns=["sentence1","sentence2","label","source"])
else:
    df_viqp = pd.DataFrame(columns=["sentence1","sentence2","label","source"])

# ---- 3.4) Merge ----
all_dfs = [dfx[["sentence1","sentence2","label","source"]]
           for dfx in [df_vnpc, df_vnpara, df_visp, df_viqp] if len(dfx) > 0]
df = pd.concat(all_dfs, ignore_index=True)
print(f"\nMerged: {len(df)} rows")

# ---- 3.5) Label cleanup ----
df["label"] = pd.to_numeric(df["label"], errors="coerce")
bad = df["label"].isna() | (~df["label"].isin([0, 1]))
if bad.sum() > 0: print(f"Removing {bad.sum()} bad labels"); df = df[~bad].copy()
df["label"] = df["label"].astype(int)
print(f"Labels: {df['label'].value_counts().to_dict()}")
print(f"Sources: {df['source'].value_counts().to_dict()}")

# ---- 3.6) Process text ----
print("\nProcessing text (norm + word-seg)...")
df["sentence1"] = df["sentence1"].map(process_text)
df["sentence2"] = df["sentence2"].map(process_text)
empty = (df["sentence1"].str.len()==0) | (df["sentence2"].str.len()==0)
if empty.any(): df = df[~empty].copy()

# ---- 3.7) Dedup ----
print("Dedup...")
pairs = df.apply(lambda r: canonical_pair(r["sentence1"], r["sentence2"]),
                 axis=1, result_type="expand")
df["a"], df["b"] = pairs[0], pairs[1]
df["pair_key"] = df["a"] + "\u0001" + df["b"]

nuniq = df.groupby("pair_key")["label"].nunique()
confl = set(nuniq[nuniq > 1].index)
if confl: print(f"  Removing {len(confl)} conflicts"); df = df[~df["pair_key"].isin(confl)].copy()
before = len(df)
df = df.drop_duplicates(subset=["pair_key","label"]).reset_index(drop=True)
print(f"  {before} → {len(df)}")

# ---- 3.8) Union-Find ----
print("Building connected components...")
all_s = pd.concat([df["a"], df["b"]]).unique()
sid = {s:i for i,s in enumerate(all_s)}
parent = list(range(len(all_s))); uf_rank = [0]*len(all_s)

def find(x):
    while parent[x] != x: parent[x]=parent[parent[x]]; x=parent[x]
    return x
def union(x,y):
    rx,ry=find(x),find(y)
    if rx==ry: return
    if uf_rank[rx]<uf_rank[ry]: parent[rx]=ry
    elif uf_rank[rx]>uf_rank[ry]: parent[ry]=rx
    else: parent[ry]=rx; uf_rank[rx]+=1

for a,b in zip(df["a"],df["b"]): union(sid[a],sid[b])
comp = {s:find(i) for s,i in sid.items()}
df["group"] = df["a"].map(comp).astype(int)
print(f"  Components: {df['group'].nunique()}")

# ---- 3.9) Split ----
print("Split train/val/test...")
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=SEED)
tv_i, ts_i = list(sgkf.split(df, df["label"], df["group"]))[0]
df_trainval = df.iloc[tv_i].reset_index(drop=True)
df_test     = df.iloc[ts_i].reset_index(drop=True)

sgkf2 = StratifiedGroupKFold(n_splits=4, shuffle=True, random_state=SEED+1)
tr_i, vl_i = list(sgkf2.split(df_trainval, df_trainval["label"], df_trainval["group"]))[0]
df_train = df_trainval.iloc[tr_i].reset_index(drop=True)
df_val   = df_trainval.iloc[vl_i].reset_index(drop=True)

for nm,dx in [("train",df_train),("val",df_val),("test",df_test)]:
    print(f"  [{nm}] n={len(dx)}, labels={dx['label'].value_counts().to_dict()}")

# ---- 3.10) Hard-neg mining ----
df_train = df_train.copy(); df_train["sample_weight"] = 1.0

if DO_HARD_NEG_MINING:
    print("\nHard-neg mining...")
    known = set(df_train["pair_key"]); pos_k = set(df_train.loc[df_train["label"]==1,"pair_key"])
    slist = pd.concat([df_train["a"],df_train["b"]]).unique().tolist()
    MAX_MINE=10000; msents = random.sample(slist, min(MAX_MINE,len(slist)))
    print(f"  Mining from {len(msents)} of {len(slist)} sents")

    vec = TfidfVectorizer(ngram_range=(1,2),min_df=2,max_df=0.95,sublinear_tf=True)
    X = vec.fit_transform(slist); s2i = {s:i for i,s in enumerate(slist)}
    nn = NearestNeighbors(n_neighbors=min(20,len(slist)),metric="cosine",algorithm="brute",n_jobs=-1)
    nn.fit(X); _, idxs = nn.kneighbors(X[[s2i[s] for s in msents]])

    mined = []; used = set(known)
    for qi,s in enumerate(msents):
        found=0
        for j in idxs[qi]:
            if j==s2i[s]: continue
            t=slist[j]; a,b=canonical_pair(s,t); key=a+"\u0001"+b
            if key in used or key in pos_k: continue
            mined.append({"sentence1":a,"sentence2":b,"label":0,"source":"mined","sample_weight":HARD_NEG_WEIGHT})
            used.add(key); found+=1
            if found>=HARD_NEG_PER_SENT: break
    print(f"  Mined {len(mined)} hard negatives")
    if mined:
        keep=["sentence1","sentence2","label","source","sample_weight"]
        df_train=pd.concat([df_train[keep],pd.DataFrame(mined)[keep]],ignore_index=True)

keep=["sentence1","sentence2","label","source","sample_weight"]
df_train=df_train[keep].copy()
df_val=df_val[["sentence1","sentence2","label","source"]].copy(); df_val["sample_weight"]=1.0
df_test=df_test[["sentence1","sentence2","label","source"]].copy(); df_test["sample_weight"]=1.0

print(f"\nFinal: train={len(df_train)}, val={len(df_val)}, test={len(df_test)}")
print(f"Train labels: {df_train['label'].value_counts().to_dict()}")

df_train.to_parquet(os.path.join(PREP_DIR,"train.parquet"),index=False)
df_val.to_parquet(os.path.join(PREP_DIR,"val.parquet"),index=False)
df_test.to_parquet(os.path.join(PREP_DIR,"test.parquet"),index=False)

del df, df_vnpc, df_vnpara, df_visp, df_viqp, df_trainval
try: del vec, X, nn, idxs
except: pass
gc.collect()
print("Data done ✓")

Loading VNPC + vnPara...


vnpc_train.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2507 [00:00<?, ? examples/s]

vnpara.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3083 [00:00<?, ? examples/s]

...

ViQP_train.jsonl: 0.00B [00:00, ?B/s]

ViQP_test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/34506 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21562 [00:00<?, ? examples/s]

  ViQP pairs: 56068

Merged: 61702 rows
Labels: {1: 59887, 0: 1815}
Sources: {'viqp': 56068, 'vnpara': 3083, 'vnpc': 2507, 'visp': 44}

Processing text (norm + word-seg)...
Dedup...
  Removing 5 conflicts
  61692 → 33509
Building connected components...
  Components: 6918
Split train/val/test...
  [train] n=20105, labels={1: 19027, 0: 1078}
  [val] n=6702, labels={1: 6342, 0: 360}
  [test] n=6702, labels={1: 6343, 0: 359}

Hard-neg mining...
  Mining from 10000 of 13681 sents
  Mined 20000 hard negatives

Final: train=40105, val=6702, test=6702
Train labels: {0: 21078, 1: 19027}
Data done ✓


In [5]:
# # CHẠY CELL NÀY TRƯỚC KHI TRAINING để kiểm tra data
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# tokenizer_check = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# config_check = AutoConfig.from_pretrained(MODEL_NAME)

# print(f"Model vocab_size:          {config_check.vocab_size}")
# print(f"Tokenizer vocab_size:      {len(tokenizer_check)}")
# print(f"max_position_embeddings:   {config_check.max_position_embeddings}")
# print(f"type_vocab_size:           {config_check.type_vocab_size}")

# # Tính max sequence length an toàn
# # RoBERTa positions bắt đầu từ padding_idx+1=2, nên max_seq = max_pos_emb - 2
# safe_max_len = config_check.max_position_embeddings - 2
# print(f"Safe MAX_LENGTH:           {safe_max_len}")
# print(f"Current MAX_LENGTH:        {MAX_LENGTH}")

# if MAX_LENGTH > safe_max_len:
#     print(f"⚠️ MAX_LENGTH={MAX_LENGTH} > safe limit {safe_max_len}! Sẽ gây lỗi position embedding!")
#     MAX_LENGTH = safe_max_len
#     print(f"✅ Đã giảm MAX_LENGTH xuống {MAX_LENGTH}")

# # Kiểm tra vài sample
# df_check = pd.read_parquet(os.path.join(PREP_DIR, "train.parquet"))
# print(f"\nChecking {min(1000, len(df_check))} samples...")
# max_tok_id = 0
# max_seq_len = 0
# for i in range(min(1000, len(df_check))):
#     enc = tokenizer_check(df_check.iloc[i]["sentence1"], df_check.iloc[i]["sentence2"],
#                           truncation=True, max_length=MAX_LENGTH, padding=False)
#     ids = enc["input_ids"]
#     max_tok_id = max(max_tok_id, max(ids))
#     max_seq_len = max(max_seq_len, len(ids))
#     if max(ids) >= config_check.vocab_size:
#         print(f"  ❌ Row {i}: token ID {max(ids)} >= vocab_size {config_check.vocab_size}")

# print(f"Max token ID seen:   {max_tok_id} (vocab_size={config_check.vocab_size})")
# print(f"Max seq length:      {max_seq_len} (max_position_embeddings={config_check.max_position_embeddings})")

# if max_tok_id < config_check.vocab_size and max_seq_len <= safe_max_len:
#     print("✅ Data looks safe!")
# else:
#     print("❌ Data has issues — see details above")

# del df_check, tokenizer_check, config_check

In [6]:
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, AutoConfig,
    TrainingArguments, Trainer, EarlyStoppingCallback, DataCollatorWithPadding,
)
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, \
    average_precision_score, classification_report

df_train = pd.read_parquet(os.path.join(PREP_DIR,"train.parquet"))
df_val   = pd.read_parquet(os.path.join(PREP_DIR,"val.parquet"))
df_test  = pd.read_parquet(os.path.join(PREP_DIR,"test.parquet"))

n0=(df_train["label"]==0).sum(); n1=(df_train["label"]==1).sum()
w0=len(df_train)/(2.*max(1,n0)); w1=len(df_train)/(2.*max(1,n1))
class_weights = torch.tensor([w0,w1],dtype=torch.float32)
print(f"Class weights: 0={w0:.3f}, 1={w1:.3f}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def tokenize_dataset(dfx):
    ds = Dataset.from_pandas(dfx, preserve_index=False)
    def tok(batch):
        enc = tokenizer(batch["sentence1"],batch["sentence2"],
                        truncation=True,max_length=MAX_LENGTH,padding=False)
        # FIX: PhoBERT/RoBERTa có type_vocab_size=1, không dùng token_type_ids
        # Tokenizer có thể trả về token_type_ids=[0,...,1,...] cho cặp câu
        # → gây "index out of bounds" trong embedding lookup
        if "token_type_ids" in enc:
            del enc["token_type_ids"]
        enc["labels"]=[int(x) for x in batch["label"]]
        enc["sample_weight"]=[float(x) for x in batch["sample_weight"]]
        return enc
    ds = ds.map(tok,batched=True,
                remove_columns=["sentence1","sentence2","label","source","sample_weight"])
    ds.set_format("torch"); return ds

train_ds=tokenize_dataset(df_train); val_ds=tokenize_dataset(df_val); test_ds=tokenize_dataset(df_test)
print(f"Tokenized: train={len(train_ds)}, val={len(val_ds)}, test={len(test_ds)}")
print(f"Columns: {train_ds.column_names}")

class WeightedPaddingCollator(DataCollatorWithPadding):
    def __call__(self, features):
        sw=[f.pop("sample_weight") for f in features] if "sample_weight" in features[0] else None
        # Đảm bảo không có token_type_ids lọt vào
        for f in features:
            f.pop("token_type_ids", None)
        batch=super().__call__(features)
        if sw: batch["sample_weight"]=torch.tensor(sw,dtype=torch.float32)
        return batch

class WeightedTrainer(Trainer):
    def __init__(self,*a,class_weights=None,**kw): super().__init__(*a,**kw); self.class_weights=class_weights
    def compute_loss(self,model,inputs,return_outputs=False,**kw):
        labels=inputs.pop("labels"); sw=inputs.pop("sample_weight",None)
        inputs.pop("token_type_ids", None)  # safety: xoá nếu còn sót
        out=model(**inputs); logits=out.logits
        cw=self.class_weights.to(logits.device) if self.class_weights is not None else None
        loss=torch.nn.CrossEntropyLoss(weight=cw,reduction="none")(logits,labels)
        if sw is not None: loss=loss*sw.to(loss.device)
        loss=loss.mean()
        return (loss,out) if return_outputs else loss

def compute_metrics(ep):
    logits,labels=ep
    probs=torch.softmax(torch.tensor(logits,dtype=torch.float32),dim=-1).numpy()
    preds=probs.argmax(axis=-1)
    mf1=f1_score(labels,preds,average="macro")
    f0=f1_score(labels,preds,pos_label=0,average="binary")
    f1=f1_score(labels,preds,pos_label=1,average="binary")
    acc=accuracy_score(labels,preds)
    try: prauc=average_precision_score(labels,probs[:,1])
    except: prauc=0.0
    return {"macro_f1":round(mf1,4),"f1_0":round(f0,4),"f1_1":round(f1,4),
            "accuracy":round(acc,4),"pr_auc_pos":round(prauc,4)}

config=AutoConfig.from_pretrained(MODEL_NAME,num_labels=2)
config.hidden_dropout_prob=DROPOUT_RATE
config.attention_probs_dropout_prob=DROPOUT_RATE
config.classifier_dropout=DROPOUT_RATE
config.id2label={0:"not_paraphrase",1:"paraphrase"}
config.label2id={"not_paraphrase":0,"paraphrase":1}
model=AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,config=config)

# FIX: đồng bộ embedding size với tokenizer vocab
model.resize_token_embeddings(len(tokenizer))
print(f"Vocab size: tokenizer={len(tokenizer)}, model={model.config.vocab_size}")

training_args=TrainingArguments(
    output_dir=OUT_DIR, learning_rate=LR, num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BS, per_device_eval_batch_size=EVAL_BS,
    gradient_accumulation_steps=GRAD_ACCUM, warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY, fp16=True, logging_steps=100,
    eval_strategy="epoch", save_strategy="epoch",
    load_best_model_at_end=True, metric_for_best_model="macro_f1",
    greater_is_better=True, save_total_limit=2, report_to="none",
    dataloader_num_workers=2, dataloader_pin_memory=True, seed=SEED,
    label_smoothing_factor=0.05,
)

trainer=WeightedTrainer(
    model=model, args=training_args, train_dataset=train_ds, eval_dataset=val_ds,
    processing_class=tokenizer, data_collator=WeightedPaddingCollator(tokenizer=tokenizer),
    compute_metrics=compute_metrics, class_weights=class_weights,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
)

ng=torch.cuda.device_count()
print(f"\nTraining: {MODEL_NAME}")
print(f"  Epochs={EPOCHS}, EffBS={TRAIN_BS*GRAD_ACCUM*ng}, LR={LR}, Dropout={DROPOUT_RATE}")
trainer.train()

Class weights: 0=0.951, 1=1.054


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.3.output.Laye

TrainOutput(global_step=5016, training_loss=0.15987159448424784, metrics={'train_runtime': 4113.1333, 'train_samples_per_second': 97.505, 'train_steps_per_second': 1.524, 'total_flos': 1.717714049919132e+16, 'train_loss': 0.15987159448424784, 'epoch': 8.0})

In [7]:
test_out=trainer.predict(test_ds)
preds=test_out.predictions.argmax(axis=-1); lbls=test_out.label_ids
cm=confusion_matrix(lbls,preds)

print("=== Test Metrics ===")
for k,v in test_out.metrics.items(): print(f"  {k}: {v}")
print(f"\nConfusion Matrix:\n  Pred →   0     1")
print(f"  Act 0: {cm[0][0]:>5} {cm[0][1]:>5}")
print(f"  Act 1: {cm[1][0]:>5} {cm[1][1]:>5}")
print(classification_report(lbls,preds,target_names=["not_para","para"]))

test_out.metrics["confusion_matrix"]=cm.tolist()
with open(os.path.join(OUT_DIR,"test_metrics.json"),"w") as f: json.dump(test_out.metrics,f,indent=2)
trainer.save_model(OUT_DIR); tokenizer.save_pretrained(OUT_DIR)

# Demo
mdl=AutoModelForSequenceClassification.from_pretrained(OUT_DIR).cuda().eval()
tok=AutoTokenizer.from_pretrained(OUT_DIR,use_fast=True)

def predict_pair(s1,s2):
    if USE_WORD_SEG: s1,s2=word_segment(norm_text(s1)),word_segment(norm_text(s2))
    x=tok(s1,s2,truncation=True,max_length=MAX_LENGTH,return_tensors="pt").to("cuda")
    with torch.no_grad(): p=torch.softmax(mdl(**x).logits,dim=-1)[0].cpu().numpy()
    return {"label":"paraphrase" if p[1]>p[0] else "not_paraphrase","p":round(float(p[1]),4)}

for s1,s2 in [
    ("Hôm nay trời mưa rất to.","Thời tiết hôm nay mưa lớn."),
    ("Giá vàng tăng mạnh.","Trận đấu tối qua rất hấp dẫn."),
    ("Thủ tướng đã họp với các bộ trưởng.","Cuộc họp của Thủ tướng với nội các đã diễn ra."),
    ("Hà Nội là thủ đô của Việt Nam.","TP.HCM là thành phố lớn nhất Việt Nam."),
]:
    r=predict_pair(s1,s2)
    print(f"\n  S1: {s1}\n  S2: {s2}\n  → {r['label']} ({r['p']})")

print("\n✅ Done!")

=== Test Metrics ===
  test_loss: 0.1292610913515091
  test_macro_f1: 0.876
  test_f1_0: 0.768
  test_f1_1: 0.9841
  test_accuracy: 0.9702
  test_pr_auc_pos: 0.9995
  test_runtime: 15.2113
  test_samples_per_second: 440.594
  test_steps_per_second: 3.484

Confusion Matrix:
  Pred →   0     1
  Act 0:   331    28
  Act 1:   172  6171
              precision    recall  f1-score   support

    not_para       0.66      0.92      0.77       359
        para       1.00      0.97      0.98      6343

    accuracy                           0.97      6702
   macro avg       0.83      0.95      0.88      6702
weighted avg       0.98      0.97      0.97      6702



Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]


  S1: Hôm nay trời mưa rất to.
  S2: Thời tiết hôm nay mưa lớn.
  → paraphrase (0.9902)

  S1: Giá vàng tăng mạnh.
  S2: Trận đấu tối qua rất hấp dẫn.
  → not_paraphrase (0.0029)

  S1: Thủ tướng đã họp với các bộ trưởng.
  S2: Cuộc họp của Thủ tướng với nội các đã diễn ra.
  → paraphrase (0.9592)

  S1: Hà Nội là thủ đô của Việt Nam.
  S2: TP.HCM là thành phố lớn nhất Việt Nam.
  → not_paraphrase (0.0005)

✅ Done!
